# Demo

In [1]:
# run for Jax GPU
import numpy as np
from agginc.jax import human_readable_dict
from agginc.jax import agginc
# from agginc.np import agginc
from jax import random
import jax.numpy as jnp
X = jnp.array([1, 2])
X.device()

StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)

## MMDAggInc

In [2]:
key = random.PRNGKey(0)
key, subkey = random.split(key)
subkeys = random.split(subkey, num=2)
X = random.uniform(subkeys[0], shape=(5000, 100))
Y = random.uniform(subkeys[1], shape=(5000, 100)) + 1

In [3]:
# compile function
output, dictionary = agginc("mmd", X, Y, return_dictionary=True)
# Numpy version (no compilation)
# output, dictionary = agginc("mmd", np.array(X), np.array(Y), return_dictionary=True)

In [4]:
# Now the function runs fast for any inputs X and Y of the compiled shaped (5000, 100)
# If the shape is changed, the function will need to be compiled again
key = random.PRNGKey(1) # different initialisation
key, subkey = random.split(key)
subkeys = random.split(subkey, num=2)
X = random.uniform(subkeys[0], shape=(5000, 100))
Y = random.uniform(subkeys[1], shape=(5000, 100)) + 1
# see speed.ipynb for detailed speed comparision between numpy, jax cpu and jax gpu 
%timeit output, dictionary = agginc("mmd", X, Y, return_dictionary=True)

22.8 ms ± 25 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
output, dictionary = agginc("mmd", X, Y, return_dictionary=True)

In [6]:
# output is a jax array consisting of either 0 or 1
output 

Array(1, dtype=int32)

In [7]:
# to convert it to an int use: 
output.item()

1

In [8]:
human_readable_dict(dictionary) # use to convert jax arrays to scalars
dictionary

{'MMDAggInc test reject': True,
 'Single test 1': {'Bandwidth': 4.58139181137085,
  'Kernel Gaussian': True,
  'MMD': 0.9013530611991882,
  'MMD quantile': 0.0017263863701373339,
  'Reject': True,
  'p-value': 0.0019960079807788134,
  'p-value threshold': 0.037924136966466904},
 'Single test 10': {'Bandwidth': 24.66503143310547,
  'Kernel Gaussian': True,
  'MMD': 0.2951463460922241,
  'MMD quantile': 0.0005236162105575204,
  'Reject': True,
  'p-value': 0.0019960079807788134,
  'p-value threshold': 0.037924136966466904},
 'Single test 2': {'Bandwidth': 5.523686408996582,
  'Kernel Gaussian': True,
  'MMD': 1.1168569326400757,
  'MMD quantile': 0.0020746372174471617,
  'Reject': True,
  'p-value': 0.0019960079807788134,
  'p-value threshold': 0.037924136966466904},
 'Single test 3': {'Bandwidth': 6.659790992736816,
  'Kernel Gaussian': True,
  'MMD': 1.2296091318130493,
  'MMD quantile': 0.0022286062594503164,
  'Reject': True,
  'p-value': 0.0019960079807788134,
  'p-value threshold':

## HSICAggInc

In [9]:
key = random.PRNGKey(0)
key, subkey = random.split(key)
subkeys = random.split(subkey, num=2)
X = random.uniform(subkeys[0], shape=(5000, 100))
Y = 0.5 * X + random.uniform(subkeys[1], shape=(5000, 100))

In [10]:
# compile function
output, dictionary = agginc("hsic", X, Y, return_dictionary=True)
# Numpy version (no compilation)
# output, dictionary = agginc("hsic", np.array(X), np.array(Y), return_dictionary=True)

In [11]:
# Now the function runs fast for any inputs X and Y of the compiled shaped (5000, 100)
# If the shape is changed, the function will need to be compiled again
key = random.PRNGKey(1) # different initialisation
key, subkey = random.split(key)
subkeys = random.split(subkey, num=2)
X = random.uniform(subkeys[0], shape=(5000, 100))
Y = 0.5 * X + random.uniform(subkeys[1], shape=(5000, 100))
# see speed.ipynb for detailed speed comparision between numpy, jax cpu and jax gpu 
%timeit output, dictionary = agginc("hsic", X, Y, return_dictionary=True)

17.9 ms ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
output, dictionary = agginc("hsic", X, Y, return_dictionary=True)

In [13]:
output

Array(1, dtype=int32)

In [14]:
output.item()

1

In [15]:
human_readable_dict(dictionary) # use to convert jax arrays to scalars
dictionary

{'HSICAggInc test reject': True,
 'Single test 1': {'Bandwidth X': 1.019727349281311,
  'Bandwidth Y': 1.1367896795272827,
  'HSIC': 7.2079468045258e-13,
  'HSIC quantile': 9.377033519239117e-14,
  'Kernel Gaussian': True,
  'Reject': True,
  'p-value': 0.0019960079807788134,
  'p-value threshold': 0.011976033449172974},
 'Single test 10': {'Bandwidth X': 16.315637588500977,
  'Bandwidth Y': 2.2735793590545654,
  'HSIC': 3.641161310952157e-05,
  'HSIC quantile': 6.56685472222307e-07,
  'Kernel Gaussian': True,
  'Reject': True,
  'p-value': 0.0019960079807788134,
  'p-value threshold': 0.011976033449172974},
 'Single test 11': {'Bandwidth X': 1.019727349281311,
  'Bandwidth Y': 4.547158718109131,
  'HSIC': 2.4181092683761563e-08,
  'HSIC quantile': 9.870516626264703e-10,
  'Kernel Gaussian': True,
  'Reject': True,
  'p-value': 0.0019960079807788134,
  'p-value threshold': 0.011976033449172974},
 'Single test 12': {'Bandwidth X': 2.039454698562622,
  'Bandwidth Y': 4.547158718109131,
 

## KSDAggInc

In [16]:
perturbation = 0.5
rs = np.random.RandomState(0)
X = rs.gamma(5 + perturbation, 5, (5000, 1))
score_gamma = lambda x, k, theta : (k - 1) / x - 1 / theta
score_X = score_gamma(X, 5, 5)
X = jnp.array(X)
score_X = jnp.array(score_X)

In [17]:
# compile function
output, dictionary = agginc("ksd", X, score_X, return_dictionary=True)
# Numpy version (no compilation)
# output, dictionary = agginc("ksd", np.array(X), np.array(score_X), return_dictionary=True)

In [18]:
# Now the function runs fast for any inputs X and score_X of the compiled shaped (5000, 1)
# If the shape is changed, the function will need to be compiled again
perturbation = 0.5
rs = np.random.RandomState(1) # different initialisation
X = rs.gamma(5 + perturbation, 5, (5000, 1))
score_gamma = lambda x, k, theta : (k - 1) / x - 1 / theta
score_X = score_gamma(X, 5, 5)
X = jnp.array(X)
score_X = jnp.array(score_X)
# see speed.ipynb for detailed speed comparision between numpy, jax cpu and jax gpu 
%timeit output, dictionary = agginc("ksd", X, score_X, return_dictionary=True)

21.5 ms ± 30.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
output, dictionary = agginc("ksd", X, score_X, return_dictionary=True)

In [20]:
output

Array(1, dtype=int32)

In [21]:
output.item()

1

In [22]:
human_readable_dict(dictionary) # use to convert jax arrays to scalars
dictionary

{'KSDAggInc test reject': True,
 'Single test 1': {'Bandwidth': 1.0,
  'KSD': 4.0504713979316875e-05,
  'KSD quantile': 0.00029450401780195534,
  'Kernel IMQ': True,
  'Reject': False,
  'p-value': 0.38522952795028687,
  'p-value threshold': 0.017964035272598267},
 'Single test 10': {'Bandwidth': 96.67843627929688,
  'KSD': 4.25097505285521e-06,
  'KSD quantile': 2.747115956935886e-07,
  'Kernel IMQ': True,
  'Reject': True,
  'p-value': 0.0019960079807788134,
  'p-value threshold': 0.017964035272598267},
 'Single test 2': {'Bandwidth': 1.661851406097412,
  'KSD': 4.388344314065762e-05,
  'KSD quantile': 8.466501458315179e-05,
  'Kernel IMQ': True,
  'Reject': False,
  'p-value': 0.113772451877594,
  'p-value threshold': 0.017964035272598267},
 'Single test 3': {'Bandwidth': 2.7617499828338623,
  'KSD': 4.6408065827563405e-05,
  'KSD quantile': 2.3729142412776127e-05,
  'Kernel IMQ': True,
  'Reject': True,
  'p-value': 0.0019960079807788134,
  'p-value threshold': 0.017964035272598267